In [43]:
import pandas as pd
from zipfile import Path as ZipPath
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import matplotlib as mpl
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    "text.usetex": True,
    "font.family": "serif",
})
import matplotlib.pyplot as plt


# wczytywanie bezpośrednio z archiwum

In [44]:
fps = {x.stem: x for x in ZipPath('Dopasowanie.zip', at='/').iterdir()}
fps                                                                

{'IRL-2.2': Path('Dopasowanie.zip', 'IRL-2.2.csv'),
 'IRL-2.1': Path('Dopasowanie.zip', 'IRL-2.1.csv'),
 'trpp': Path('Dopasowanie.zip', 'trpp.csv')}

In [45]:
for k,v in fps.items():
    print(k, v.exists())

IRL-2.2 True
IRL-2.1 True
trpp True


In [46]:
%%time

dfs = {}
for k,v in fps.items():
    with v.open() as f:
        dfs[k] = pd.read_csv(f, parse_dates=['UTC'], index_col='UTC')


CPU times: user 1.23 s, sys: 15.1 ms, total: 1.24 s
Wall time: 1.25 s


# połączenie w jednego dataframe


In [47]:
df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"]
    ],
    axis=1
).sort_index()


In [48]:
print(df.columns.tolist())
print(dfs.keys())

['IRL-2.1', 'IRL-2.2', 'PLC_time', 'PAR', 'PK1', 'PK2', 'PK3', 'PK4', 'PK5', 'PK6', 'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'PB6']
dict_keys(['IRL-2.2', 'IRL-2.1', 'trpp'])


In [49]:

%%time
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

# GÓRA: IRL (tylko raz)
df.plot(
    ax=ax1,
    y=["IRL-2.1", "IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

# DÓŁ: TRPP
df.plot(
    ax=ax2,
    y=["PAR", "PK2", "PK6", "PB1"],
    grid=True,
    marker=".",
)

# ax1.legend()
# ax2.legend()
# plt.tight_layout()
# fig.savefig(f"{outdir}/całośc.png", format="png")  # bez tight

# ax1.legend()


CPU times: user 744 ms, sys: 19.9 ms, total: 764 ms
Wall time: 768 ms


<Axes: xlabel='UTC'>

In [50]:
intervals = {
    "zrzuty": slice("2025-12-08 19:55:30", "2025-12-08 21:30:30"),
    "zrzut01": slice("2025-12-08 19:55:30", "2025-12-08 20:03:30"),
    "zrzut02": slice("2025-12-08 20:07:30", "2025-12-08 20:11:30"),
    "zrzut03": slice("2025-12-08 20:15:30", "2025-12-08 20:19:30"),
    "zrzut04": slice("2025-12-08 20:23:30", "2025-12-08 20:27:30"),
    "zrzut05": slice("2025-12-08 20:31:30", "2025-12-08 20:35:30"),
    "zrzut06": slice("2025-12-08 20:39:30", "2025-12-08 20:43:30"),
    "zrzut07": slice("2025-12-08 20:47:30", "2025-12-08 20:51:30"),
    "zrzut08": slice("2025-12-08 20:55:30", "2025-12-08 20:59:30"),
    "zrzut09": slice("2025-12-08 21:00:30", "2025-12-08 21:07:30"),
    "zrzut10": slice("2025-12-08 21:08:30", "2025-12-08 21:12:30"),
    "zrzut11": slice("2025-12-08 21:12:30", "2025-12-08 21:18:30"),
}

[(k, v.start, v.stop) for k, v in intervals.items()]


[('zrzuty', '2025-12-08 19:55:30', '2025-12-08 21:30:30'),
 ('zrzut01', '2025-12-08 19:55:30', '2025-12-08 20:03:30'),
 ('zrzut02', '2025-12-08 20:07:30', '2025-12-08 20:11:30'),
 ('zrzut03', '2025-12-08 20:15:30', '2025-12-08 20:19:30'),
 ('zrzut04', '2025-12-08 20:23:30', '2025-12-08 20:27:30'),
 ('zrzut05', '2025-12-08 20:31:30', '2025-12-08 20:35:30'),
 ('zrzut06', '2025-12-08 20:39:30', '2025-12-08 20:43:30'),
 ('zrzut07', '2025-12-08 20:47:30', '2025-12-08 20:51:30'),
 ('zrzut08', '2025-12-08 20:55:30', '2025-12-08 20:59:30'),
 ('zrzut09', '2025-12-08 21:00:30', '2025-12-08 21:07:30'),
 ('zrzut10', '2025-12-08 21:08:30', '2025-12-08 21:12:30'),
 ('zrzut11', '2025-12-08 21:12:30', '2025-12-08 21:18:30')]

In [51]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

for name, sl in intervals.items():

    df_cut = df.loc[sl]
    if name == "zrzuty":
        continue
    if df_cut.empty:
        print(f"{name}: pusty zakres")
        continue

    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        figsize=(16, 12),
        sharex=True
    )

    # ===== GÓRA: IRL =====
    df_cut.plot(
        ax=ax1,
        y=["IRL-2.1"],
        logy=True,
        grid=True,
        marker=".",
    )

    # ===== DÓŁ: PK6 =====
    df_cut.plot(
        ax=ax2,
        y=["PK6"],
        grid=True,
        marker=".",
    )

    # format osi X
    ax1.set_title(f"{name} | {sl.start} → {sl.stop}")
    ax2.set_xlabel("UTC")

    plt.tight_layout()
    plt.show()



/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_19691/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [52]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.1"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.savefig 
plt.show()


/tmp/ipykernel_19691/3702980905.py:22: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [53]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()


/tmp/ipykernel_19691/4119963235.py:21: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
   
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]
    

In [55]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
    "zrzut02": ("2025-12-08 20:08:40", "2025-12-08 20:09:45"),
    "zrzut03": ("2025-12-08 20:17:15", "2025-12-08 20:18:05"),
    "zrzut04": ("2025-12-08 20:25:05", "2025-12-08 20:26:10"),
    "zrzut05": ("2025-12-08 20:32:48", "2025-12-08 20:33:43"),
    "zrzut06": ("2025-12-08 20:41:08", "2025-12-08 20:41:53"),
    "zrzut07": ("2025-12-08 20:49:13", "2025-12-08 20:50:08"),
    "zrzut08": ("2025-12-08 20:56:33", "2025-12-08 20:57:33"),
    "zrzut09": ("2025-12-08 21:02:50", "2025-12-08 21:03:40"),
    "zrzut10": ("2025-12-08 21:09:22", "2025-12-08 21:10:12"),
    "zrzut11": ("2025-12-08 21:14:59", "2025-12-08 21:15:34"),
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]

In [56]:
t

Index([         0.0,  0.023238897,  0.099303007,  0.119539976,  0.199012995,
        0.220527888,  0.299893856,  0.320500851,  0.400672913,  0.423434973,
       ...
       35.500520945, 35.521129847, 35.600239039,  35.62114501,  35.70023799,
       35.720597983, 35.799998045, 35.820545912, 35.899873972, 35.922618866],
      dtype='float64', name='UTC', length=754)

In [57]:

# def lin_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu liniowego
#     y = a*t+b
#     return y

# def par_fit(t,x1,x2,a,b,c,T,x):    # funkcja do segmentu paraboli
#     d=-2*c*x1
#     f = c*x2**2 + d*x2
#     y = c*t**2 *d*t + f
#     return y

# # def exp_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu exp
# #     d=-2*c*x1
# #     f = c*x2**2 + d*x2
# #     g = (c*x2**2 + d*x2+f)/np.exp(x2/T)
# #     y = np.exp(t/T)*g
# #     return y

# def exp_fit(t, a, T):   # funkcja do segmentu exp
#     y = np.exp(t/T) *a
#     return y



In [58]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

b_i   = np.array([2.43e-4, 1.363e-3, 1.203e-3, 2.605e-3, 8.19e-4, 1.67e-4])
lambda_i = np.array([1.27e-2, 3.17e-2, 1.15e-1, 3.11e-1, 1.40,   3.87   ])  # [1/s]
B = b_i.sum()
L = 146e-6

def exp_fit(t, a, T):
    return np.exp(t/T) * a   

def rho_T(T_hat, L,B, b_i, lambda_i):
    suma = np.sum((b_i) / ((1 /T_hat)+lambda_i))
    nie_suma = 1/(T_hat*B)
    rho = nie_suma * (L+ suma)
    return rho

def sigma_rho_T(sigma_T, L,B, b_i, lambda_i, T_hat):
    suma = np.sum((b_i * lambda_i) / (B * (lambda_i*T_hat + 1)**2))
    nie_suma = L/((T_hat**2)*B)
    sigma_rho = (nie_suma + suma)*sigma_T
    return sigma_rho

def pm(val, err, vfmt="{:.2f}", efmt="{:.2f}"):
    return f"{vfmt.format(val)} ± {efmt.format(err)}"

wyniki = []

fig, ax = plt.subplots(figsize=(20,12))

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1, ["IRL-2.1"]].dropna()
    if maly_df.empty:
        print(name, "-> pusty wycinek")
        continue

    t = (maly_df.index - maly_df.index[0]).total_seconds().to_numpy()
    y = maly_df["IRL-2.1"].to_numpy()

    if not (np.isfinite(t).all() and np.isfinite(y).all()):
        print(name, "-> NaN/inf w t lub y")
        continue

    # --- FIT DLA TEGO WYCINKA ---
    try:
        popt, pcov = curve_fit(exp_fit, t, y, p0=[y[0], 60], maxfev=10000)
    except Exception as e:
        print(name, "-> curve_fit error:", e)
        continue

    sigma_a, sigma_T = np.sqrt(np.diag(pcov))
    a_hat, T_hat = popt

    r  = rho_T(T_hat, L, B, b_i, lambda_i)
    dr = sigma_rho_T(sigma_T, L, B, b_i, lambda_i, T_hat)

    wyniki.append([name, T_hat, sigma_T, r, dr])

    ax.plot(t, y, marker='.', ls='None', ms=2, alpha=0.6, label=f"{name} data")
    ax.plot(t, exp_fit(t, *popt), lw=2, label=f"{name} fit")

#  - - - format wykresu - - -
ax.set_yscale('log')
ax.grid(True, which='both', alpha=0.3)
ax.legend(ncol=2, fontsize=18)
ax.set_xlabel("Czas od początku zrzutu s")
ax.set_ylabel("IRL-2.1 Flux, cps")

# --- tabela wyników ---
import pandas as pd

fig, ax = plt.subplots(figsize=(6.3, 3.9), constrained_layout=True)

# ... rysowanie ...
print(pd.DataFrame(wyniki, columns=["zrzut", "T_hat [s]", "sigma_T [s]", "a_hat", "sigma_a"]  ))


# jeśli chcesz podgląd, to rysuj osobno w kolejnej komórce
# display(fig) / plt.show()


      zrzut   T_hat [s]  sigma_T [s]     a_hat   sigma_a
0   zrzut01  211.668029     0.048407  0.051996  0.000010
1   zrzut02   76.472193     0.108678  0.118643  0.000125
2   zrzut03   61.730370     0.082127  0.138612  0.000131
3   zrzut04   76.962828     0.134844  0.118081  0.000154
4   zrzut05   63.525941     0.132786  0.135809  0.000203
5   zrzut06   63.445978     0.111244  0.135931  0.000170
6   zrzut07   71.399725     0.119539  0.124801  0.000153
7   zrzut08   76.697220     0.109663  0.118385  0.000126
8   zrzut09   71.684540     0.105436  0.124438  0.000134
9   zrzut10   67.618561     0.139202  0.129845  0.000194
10  zrzut11   67.899713     0.096464  0.129455  0.000133


Funkcja: scipy.optimize.curve_fit – nieliniowe MNK (Levenberg–Marquardt domyślnie).
exp_fit – Twoja funkcja modelu 
f(t; a,T) zwracająca wartościdopasowania dla wektora t
t – wektor/seria argumentu niezależnego (np. czas).
y – wektor danych obserwowanych (zmienna zależna).
p0=[y.iloc[0], 60] – zgad startowy parametrów [alfa i T ]
maxfev=10000 – maks. liczba wywołań funkcji modelu (zwiększa szansę zbieżności).

In [59]:
popt, pcov = curve_fit(exp_fit, t,y, p0=[y[0], 60], maxfev=10000)
popt, np.sqrt(np.diag(pcov)) # pierwszy to parametry a i T drugi to ich niepewności

(array([4.14602492e+06, 6.78997134e+01]),
 array([2.02755060e+03, 9.64644453e-02]))

popt – wektor estymowanych parametrów
pcov – przybliżona macierz kowariancji estymatorów (Podaje nam wartość z moich danych odchylenia standardowego, ale zwraca jest jako simga^2 więc trzeba ją wstaić pod pierwiastek drugiego stopnia )

In [60]:
sigma_a, sigma_T = np.sqrt(np.diag(pcov))   # Δa, ΔT
a_hat, T_hat = popt                         # a, T

In [61]:
print(f"a = {a_hat:.6g} ± {sigma_a:.2g}")
print(f"T = {T_hat:.6g} ± {sigma_T:.2g} [h]")


a = 4.14602e+06 ± 2e+03
T = 67.8997 ± 0.096 [h]


In [62]:
fig, ax = plt.subplots(figsize=(20,12))
ax.plot(t, y, marker='.', label='data')
ax.plot(t, exp_fit(t, *popt), 'r-', label='fit')
ax.set_yscale('log')
ax.grid(True, which='both', alpha=0.3)
ax.legend()

In [63]:
# plt.figure(figsize=(20,12))
# plt.plot(t, y, marker='.', label=cn)
# plt.plot(t, exp_fit(t, *popt), 'r-', label='fit')
# plt.axvline(x= popt[0], color='gray', linestyle='--')
# plt.axvline(x= popt[1], color='gray', linestyle=':')
# plt.grid(True, alpha=0.3)
# plt.legend()
# plt.xlabel('Czas, s')
# plt.ylabel('Liczba zliczen, cps')
# plt.tight_layout()
# plt.show()
# # (opcjonalnie) szybka diagnostyka dopasowania
# res = y - exp_fit(t, *popt)
# print("Parametry (x1, x2_raw, a, b, p, c, T) =", [f"{v:.3g}" for v in popt])
# print(f"RMSE = {np.sqrt(np.mean(res**2)):.6g}, MAE = {np.mean(np.abs(res)):.6g}")


Numer telefonu do tomka